In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

# Reading images and converting all to array

In [2]:
# Initial processing, uncomment on first run

training_data = [] # Data array

img_size = 224
DataSource = "Dataset/" # Dataset source
Categories = ["With_Face_Mask", "Without_Face_Mask"] # Categories

# Function to loop through and 
def create_training_data():
    for category in Categories:
        path = os.path.join(DataSource, category)
        category_index = Categories.index(category)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))
                new_arr = cv2.resize(img_arr, (img_size, img_size)) # Resize image from 1024 x 1024 to 224 x 224
                training_data.append([new_arr, category_index]) # Append image data with category label to data array
            except Exception as e:
                pass
        

In [3]:
# Initial processing, uncomment on first run

create_training_data()

In [4]:
print(len(training_data))

1950


In [5]:
# Initial processing, uncomment on first run

import random

random.shuffle(training_data) # Shuffle to not let deep learning follow the sequence

In [6]:
import pickle
X = []
y = []

In [7]:
# Initial processing, uncomment on first run

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 3)

In [8]:
# Initial processing, uncomment on first run

# Normalize data
X = X / 255.0;

In [9]:
# Initial processing, uncomment on first run

Y = np.array(y) # Convert list into numpy array

In [10]:
# Initial processing, uncomment on first run

# Store data into pickle
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(Y, pickle_out)
pickle_out.close()

In [11]:
# Load preprocessed data from pickle
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
Y = pickle.load(pickle_in)

# Deep learning model for training - Transfer learning

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [13]:
model = tf.keras.applications.mobilenet.MobileNet() ## Pre-trained model

# Transfer learning - Tuning, weights will start from the last checkpoint

In [14]:
base_input = model.layers[0].input

In [15]:
base_output = model.layers[-4].output

In [16]:
flat_layer = layers.Flatten()(base_output)
final_output = layers.Dense(1)(flat_layer)
final_ouput = layers.Activation('sigmoid')(final_output)

In [17]:
new_model = keras.Model(inputs = base_input, outputs = final_output)

# Settings for binary classification (With face mask/Without face mask)

In [18]:
new_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [19]:
new_model.fit(X,Y,epochs = 1,validation_split = 0.1)

55/55 [==============================] - 70s 1s/step - loss: 0.2686 - accuracy: 0.9795 - val_loss: 7.9765 - val_accuracy: 0.4769
